In [1]:
import captum
import os
import argparse
import pickle
import torch
import torch_geometric
from torch_geometric.nn import GINConv, global_add_pool,GCNConv
from torch_geometric.loader import DataLoader
from torch_geometric.nn.models import MLP
from ogb.graphproppred import PygGraphPropPredDataset, Evaluator
from ogb.graphproppred.mol_encoder import AtomEncoder
import pandas as pd
import copy
import numpy as np
import random
from tqdm import tqdm
from rdkit import Chem
from rdkit.Chem import Draw
from ogb.utils.mol import smiles2graph
from torch_geometric.data import Data

/Users/tarun/0_REPOS/Explain_ai/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# from ogb.graphproppred import PygGraphPropPredDataset

# # Load the dataset
# dataset = PygGraphPropPredDataset(name="ogbg-molhiv")
# # Get a sample (e.g., the first sample)
# sample = dataset[0]

/Users/tarun/0_REPOS/Explain_ai/.venv/lib/python3.12/site-packages/ogb/graphproppred/dataset_pyg.py:68: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.data, self.slices 

In [ ]:
dataset.num_tasks
dataset.

1

In [2]:
hiv_data = pd.read_csv("~/Downloads/HIV.csv")

In [3]:
from torch_geometric.data import InMemoryDataset

class MoleculeDataset(InMemoryDataset):
    def __init__(self, data_list, transform=None):
        # Initialize the dataset with InMemoryDataset superclass
        super().__init__(None, transform)
        
        # Collate the data list into data and slices (PyTorch Geometric expects this format)
        self.data, self.slices = self.collate(data_list)
    
    @property
    def processed_file_names(self):
        # Since we're storing data in memory, we can just return an empty list
        return []

    def process(self):
        # No need to process files, as we are already providing data directly
        pass


In [4]:
data_list = []
# data = None
for i,j in enumerate(list(hiv_data['smiles'])):
    graph_data_sample = smiles2graph(j)
    # Convert edge_index to a tensor and ensure it's in the correct shape
    edge_index = torch.tensor(graph_data_sample['edge_index'], dtype=torch.long)
    # Convert edge_feat and node_feat to tensors
    edge_attr = torch.tensor(graph_data_sample['edge_feat'], dtype=torch.long)
    x = torch.tensor(graph_data_sample['node_feat'], dtype=torch.long)
    # Convert the target column from the DataFrame to a tensor
    y = torch.tensor(hiv_data['HIV_active'][i], dtype=torch.long).reshape(1,1)
    num_nodes = torch.tensor(graph_data_sample['num_nodes'], dtype=torch.long)

    data = Data(edge_index=edge_index, edge_attr=edge_attr, x=x, y=y, num_nodes=num_nodes)
    data_list.append(data)

# loader = DataLoader(data_list, batch_size=32)
# random.shuffle(data_list)

[00:06:09] WARNING: not removing hydrogen atom without neighbors
[00:06:09] WARNING: not removing hydrogen atom without neighbors


In [ ]:
# data_list[0]

Data(x=[19, 9], edge_index=[2, 40], edge_attr=[40, 3], y=[1, 1], num_nodes=19)

In [ ]:
# dataset = MoleculeDataset(data_list)


Processing...
Done!


In [ ]:
# from torch_geometric.loader import DataLoader

# # Create a DataLoader
# batch_size = 4
# dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# # Example: Iterating through the DataLoader
# for batch in dataloader:
#     print(batch)

DataBatch(x=[76, 9], edge_index=[2, 164], edge_attr=[164, 3], y=[4, 1], num_nodes=76, batch=[76], ptr=[5])
DataBatch(x=[69, 9], edge_index=[2, 144], edge_attr=[144, 3], y=[4, 1], num_nodes=69, batch=[69], ptr=[5])
DataBatch(x=[95, 9], edge_index=[2, 212], edge_attr=[212, 3], y=[4, 1], num_nodes=95, batch=[95], ptr=[5])
DataBatch(x=[91, 9], edge_index=[2, 202], edge_attr=[202, 3], y=[4, 1], num_nodes=91, batch=[91], ptr=[5])
DataBatch(x=[104, 9], edge_index=[2, 226], edge_attr=[226, 3], y=[4, 1], num_nodes=104, batch=[104], ptr=[5])
DataBatch(x=[93, 9], edge_index=[2, 196], edge_attr=[196, 3], y=[4, 1], num_nodes=93, batch=[93], ptr=[5])
DataBatch(x=[83, 9], edge_index=[2, 180], edge_attr=[180, 3], y=[4, 1], num_nodes=83, batch=[83], ptr=[5])
DataBatch(x=[65, 9], edge_index=[2, 140], edge_attr=[140, 3], y=[4, 1], num_nodes=65, batch=[65], ptr=[5])
DataBatch(x=[105, 9], edge_index=[2, 232], edge_attr=[232, 3], y=[4, 1], num_nodes=105, batch=[105], ptr=[5])
DataBatch(x=[98, 9], edge_index

In [5]:

sample_size = int(0.7 * len(data_list)) 

train_sample = data_list[:sample_size]
valid_sample = data_list[sample_size:]

# 70/30 spilt train/valid

train_dataset = MoleculeDataset(train_sample)
val_dataset = MoleculeDataset(valid_sample)

Processing...
Done!
Processing...
Done!


In [ ]:
# dataset.num_tasks = 1

In [6]:
def set_seeds(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
set_seeds(34)

In [8]:
config = {
 'device': 'cuda',
 'dataset_id': 'ogbg-molhiv',
 'num_layers': 2,
 'hidden_dim': 64,
 'dropout': 0.5,
 'learning_rate': 0.001,
 'epochs': 25,
 'batch_size': 32,
 'weight_decay': 1e-6
}

In [1]:
Evaluator(name=config["dataset_id"])

NameError: name 'Evaluator' is not defined

In [9]:
batch_size = 32
evaluator = Evaluator(name=config["dataset_id"])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [10]:
for i, batch in enumerate(train_loader):
    print(f' - Batch {i}: {batch}')

print('\nValidation loader:')
for i, batch in enumerate(val_loader):
    print(f' - Batch {i}: {batch}')

 - Batch 0: DataBatch(x=[800, 9], edge_index=[2, 1708], edge_attr=[1708, 3], y=[32, 1], num_nodes=800, batch=[800], ptr=[33])
 - Batch 1: DataBatch(x=[700, 9], edge_index=[2, 1480], edge_attr=[1480, 3], y=[32, 1], num_nodes=700, batch=[700], ptr=[33])
 - Batch 2: DataBatch(x=[778, 9], edge_index=[2, 1658], edge_attr=[1658, 3], y=[32, 1], num_nodes=778, batch=[778], ptr=[33])
 - Batch 3: DataBatch(x=[789, 9], edge_index=[2, 1724], edge_attr=[1724, 3], y=[32, 1], num_nodes=789, batch=[789], ptr=[33])
 - Batch 4: DataBatch(x=[733, 9], edge_index=[2, 1558], edge_attr=[1558, 3], y=[32, 1], num_nodes=733, batch=[733], ptr=[33])
 - Batch 5: DataBatch(x=[808, 9], edge_index=[2, 1760], edge_attr=[1760, 3], y=[32, 1], num_nodes=808, batch=[808], ptr=[33])
 - Batch 6: DataBatch(x=[858, 9], edge_index=[2, 1852], edge_attr=[1852, 3], y=[32, 1], num_nodes=858, batch=[858], ptr=[33])
 - Batch 7: DataBatch(x=[977, 9], edge_index=[2, 2094], edge_attr=[2094, 3], y=[32, 1], num_nodes=977, batch=[977], pt

In [11]:
## architecture from stanford project by william bruns
# computes a node embedding using GINConv layers, then uses pooling to predict graph level properties
class GINGraphPropertyModel(torch.nn.Module):
    def __init__(self, hidden_dim, output_dim, num_layers, dropout_p):
      super(GINGraphPropertyModel, self).__init__()
      # fields used for computing node embedding
      self.node_encoder = AtomEncoder(hidden_dim)

      self.convs = torch.nn.ModuleList(
          [torch_geometric.nn.conv.GINConv(MLP([hidden_dim, hidden_dim, hidden_dim])) for idx in range(0, num_layers)]
      )
      self.bns = torch.nn.ModuleList(
          [torch.nn.BatchNorm1d(num_features = hidden_dim) for idx in range(0, num_layers - 1)]
      )
      self.dropout_p = dropout_p
      # end fields used for computing node embedding
      # fields for graph embedding
      self.pool = global_add_pool
      self.linear_hidden = torch.nn.Linear(hidden_dim, hidden_dim)
      self.linear_out = torch.nn.Linear(hidden_dim, output_dim)
      # end fields for graph embedding
    def reset_parameters(self):
      for conv in self.convs:
        conv.reset_parameters()
      for bn in self.bns:
        bn.reset_parameters()
      self.linear_hidden.reset_parameters()
      self.linear_out.reset_parameters()
    def forward(self, batched_data):
      x, edge_index, batch = batched_data.x, batched_data.edge_index, batched_data.batch
      # compute node embedding
      x = self.node_encoder(x)
      for idx in range(0, len(self.convs)):
        x = self.convs[idx](x, edge_index)
        if idx < len(self.convs) - 1:
          x = self.bns[idx](x)
          x = torch.nn.functional.relu(x)
          x = torch.nn.functional.dropout(x, self.dropout_p, training=self.training)
      # note x is raw logits, NOT softmax'd
      # end computation of node embedding
      # convert node embedding to a graph level embedding using pooling
      x = self.pool(x, batch)
      x = torch.nn.functional.dropout(x, self.dropout_p, training=self.training)
      # transform the graph embedding to the output dimension
      # MLP after graph embed ensures we are not requiring the raw pooled node embeddings to be linearly separable
      x = self.linear_hidden(x)
      x = torch.nn.functional.relu(x)
      x = torch.nn.functional.dropout(x, self.dropout_p, training=self.training)
      out = self.linear_out(x)
      return out

In [12]:
# can be used with multiple task outputs (like for molpcba) or single task output;
# and supports using just the first output of a multi-task model if applied to a single task (for pretraining molpcba and transferring to molhiv)
def train(model, device, data_loader, optimizer, loss_fn):
  model.train()
  for step, batch in enumerate(tqdm(data_loader, desc="Training batch")):
    batch = batch.to(device)
    if batch.x.shape[0] != 1 and batch.batch[-1] != 0:
      # ignore nan targets (unlabeled) when computing training loss.
      non_nan = batch.y == batch.y
      loss = None
      optimizer.zero_grad()
      out = model(batch)
      non_nan = non_nan[:min(non_nan.shape[0], out.shape[0])]
      batch_y = batch.y[:out.shape[0], :]
      # for crudely adapting multitask models to single task data
      if batch.y.shape[1] == 1:
        out = out[:, 0]
        batch_y = batch_y[:, 0]
        non_nan = batch_y == batch_y
        loss = loss_fn(out[non_nan].reshape(-1, 1)*1., batch_y[non_nan].reshape(-1, 1)*1.)
      else:
        loss = loss_fn(out[non_nan], batch_y[non_nan])
      loss.backward()
      optimizer.step()
  return loss.item()

def eval(model, device, loader, evaluator, save_model_results=False, save_filename=None):
  model.eval()
  y_true = []
  y_pred = []
  for step, batch in enumerate(tqdm(loader, desc="Evaluation batch")):
      batch = batch.to(device)
      if batch.x.shape[0] == 1:
          pass
      else:
          with torch.no_grad():
              pred = model(batch)
              # for crudely adapting multitask models to single task data
              if batch.y.shape[1] == 1:
                pred = pred[:, 0]
              batch_y = batch.y[:min(pred.shape[0], batch.y.shape[0])]
              y_true.append(batch_y.view(pred.shape).detach().cpu())
              y_pred.append(pred.detach().cpu())
  y_true = torch.cat(y_true, dim=0).numpy()
  y_pred = torch.cat(y_pred, dim=0).numpy()
  input_dict = {"y_true": y_true.reshape(-1, 1) if batch.y.shape[1] == 1 else y_true, "y_pred": y_pred.reshape(-1, 1) if batch.y.shape[1] == 1 else y_pred}
  if save_model_results:
      single_task = len(y_true.shape) == 1 or y_true.shape[1] == 1
      if single_task:
          data = {
              'y_pred': y_pred.squeeze(),
              'y_true': y_true.squeeze()
          }
          pd.DataFrame(data=data).to_csv('ogbg_graph_' + save_filename + '.csv', sep=',', index=False)
      else:
          num_tasks = y_true.shape[1]
          for task_idx in range(num_tasks):
              data = {
                  'y_pred': y_pred[:, task_idx].squeeze(),
                  'y_true': y_true[:, task_idx].squeeze()
              }
              pd.DataFrame(data=data).to_csv('ogbg_graph_' + save_filename + f'_task_{task_idx}.csv', sep=',', index=False)
  return evaluator.eval(input_dict)

def combined_eval(gin_model, gcn_model, device, loader, evaluator):
    gin_model.eval()
    gcn_model.eval()

    y_true = []
    y_pred = []

    for step, batch in enumerate(tqdm(loader, desc="Evaluation batch")):
      batch = batch.to(device)
      if batch.x.shape[0] == 1:
        pass
      else:
        with torch.no_grad():
          pred1 = gin_model(batch)
          pred2 = gcn_model(batch)
          pred = (pred1 + pred2) / 2

          if batch.y.shape[1] == 1:
            pred = pred[:, 0]
          batch_y = batch.y[:min(pred.shape[0], batch.y.shape[0])]
          y_true.append(batch_y.view(pred.shape).detach().cpu())
          y_pred.append(pred.detach().cpu())

    y_true = torch.cat(y_true, dim=0).numpy()
    y_pred = torch.cat(y_pred, dim=0).numpy()
    input_dict = {"y_true": y_true.reshape(-1, 1) if batch.y.shape[1] == 1 else y_true, "y_pred": y_pred.reshape(-1, 1) if batch.y.shape[1] == 1 else y_pred}
    # if save_model_results:
    #     single_task = len(y_true.shape) == 1 or y_true.shape[1] == 1
    #     if single_task:
    #         data = {
    #             'y_pred': y_pred.squeeze(),
    #             'y_true': y_true.squeeze()
    #         }
    #         pd.DataFrame(data=data).to_csv('ogbg_graph_' + save_filename + '.csv', sep=',', index=False)
    #     else:
    #         num_tasks = y_true.shape[1]
    #         for task_idx in range(num_tasks):
    #             data = {
    #                 'y_pred': y_pred[:, task_idx].squeeze(),
    #                 'y_true': y_true[:, task_idx].squeeze()
    #             }
    #             pd.DataFrame(data=data).to_csv('ogbg_graph_' + save_filename + f'_task_{task_idx}.csv', sep=',', index=False)
    return evaluator.eval(input_dict)


# Run combined evaluation on the validation set
# combined_val_performance = combined_eval(gin_model, gcn_model, device, valid_loader, evaluator)
# print("Combined Validation Performance:", combined_val_performance)



In [15]:
num_tasks = 1
model = GINGraphPropertyModel(config['hidden_dim'], num_tasks, config['num_layers'], config['dropout']).to(device)
print(f"parameter count: {sum(p.numel() for p in model.parameters())}")
model.reset_parameters()

optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'], weight_decay=config['weight_decay'])
loss_fn = torch.nn.BCEWithLogitsLoss()
best_model = None
best_valid_metric_at_save_checkpoint = 0
best_train_metric_at_save_checkpoint = 0

for epoch in range(1, 1 + config["epochs"]):
  if epoch == 10:
    # reduce learning rate at this point
    optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate']*0.5, weight_decay=config['weight_decay'])
  loss = train(model, device, train_loader, optimizer, loss_fn)
  train_perf = eval(model, device, train_loader, evaluator)
  val_perf = eval(model, device, val_loader, evaluator)
  # if not args.hide_test_metric:
  #     # not necessary as output unused during train loop but needed for reproduciblility as affects number of random number generations, affecting ability to generate previously observed outputs depending on seed
  #     test_perf = eval(model, device, test_loader, evaluator)
  eval_metric = 'rocauc' 
  train_metric, valid_metric = train_perf[eval_metric], val_perf[eval_metric]
  if valid_metric >= best_valid_metric_at_save_checkpoint and train_metric >= best_train_metric_at_save_checkpoint:
    print(f"New best validation score: {valid_metric} ({eval_metric}) without training score regression")
    best_valid_metric_at_save_checkpoint = valid_metric
    best_train_metric_at_save_checkpoint = train_metric
    best_model = copy.deepcopy(model)
  print(f'Dataset {config["dataset_id"]}, ',
    f'Epoch: {epoch}, ',
    f'Train: {train_metric:.6f} ({eval_metric}), ',
    f'Valid: {valid_metric:.6f} ({eval_metric}), ')

with open(f"best_{config['dataset_id']}_gin_model_{config['num_layers']}_layers_{config['hidden_dim']}_hidden.pkl", "wb") as f:
  pickle.dump(best_model, f)

train_metric = eval(best_model, device, train_loader, evaluator)[eval_metric]
valid_metric = eval(best_model, device, val_loader, evaluator, save_model_results=True, save_filename=f"gin_{config['dataset_id']}_valid")[eval_metric]
test_metric = None

print(f'Best model for {config["dataset_id"]} (eval metric {eval_metric}): ',
      f'Train: {train_metric:.6f}, ',
      f'Valid: {valid_metric:.6f} ')
print(f"parameter count: {sum(p.numel() for p in best_model.parameters())}")

parameter count: 32385


Training batch:   0%|          | 0/900 [00:00<?, ?it/s]

Evaluation batch: 100%|██████████| 386/386 [00:01<00:00, 245.01it/s]


New best validation score: 0.5656637059322489 (rocauc) without training score regression
Dataset ogbg-molhiv,  Epoch: 1,  Train: 0.675529 (rocauc),  Valid: 0.565664 (rocauc), 


Evaluation batch: 100%|██████████| 386/386 [00:01<00:00, 210.47it/s]


...

New best validation score: 0.7299201811583095 (rocauc) without training score regression
Dataset ogbg-molhiv,  Epoch: 24,  Train: 0.824356 (rocauc),  Valid: 0.729920 (rocauc), 


Evaluation batch: 100%|██████████| 386/386 [00:01<00:00, 230.93it/s]


New best validation score: 0.7358620893138949 (rocauc) without training score regression
Dataset ogbg-molhiv,  Epoch: 25,  Train: 0.825568 (rocauc),  Valid: 0.735862 (rocauc), 


Evaluation batch: 100%|██████████| 386/386 [00:01<00:00, 236.08it/s]


Best model for ogbg-molhiv (eval metric rocauc):  Train: 0.825568,  Valid: 0.735862 
parameter count: 32385


In [16]:
def smile_to_hiv_prob(smile,best_model):
  graph_data_sample = smiles2graph(smile)

  single_molecule_like = Data(
    x=torch.tensor(graph_data_sample['node_feat']),
    edge_index=torch.tensor(graph_data_sample['edge_index']),
    edge_attr=torch.tensor(graph_data_sample['edge_feat']),
    num_nodes=torch.tensor(graph_data_sample['num_nodes']) )

  single_molecule_like = single_molecule_like.to(device) # device can be 'cuda' or 'cpu'
  best_model.eval()

    # use single_molecule_like with best model
  with torch.no_grad():
    logits = best_model(single_molecule_like)
    probabilities = torch.sigmoid(logits)
    # print("Predicted probabilities:", probabilities)

  return probabilities


In [17]:

# Sample SMILES data
c_active = hiv_data[hiv_data['activity'] =="CA"]
c_inactive = hiv_data[hiv_data['activity'] =="CI"]


smiles_data = list(c_active['smiles'].head(2)) + list(c_inactive['smiles'].head(2))
activity_data = list(c_active['activity'].head(2)) + list(c_inactive['activity'].head(2))

# Convert SMILES to RDKit Molecule objects
molecules = [Chem.MolFromSmiles(smiles) for smiles in smiles_data]

# Display molecules in a grid
img = Draw.MolsToGridImage(molecules, molsPerRow=2, subImgSize=(300, 300), legends=activity_data, maxMols=4)
print(hiv_data.iloc[0:4,])

                                              smiles activity  HIV_active
0  CCC1=[O+][Cu-3]2([O+]=C(CC)C1)[O+]=C(CC)CC(CC)...       CI           0
1  C(=Cc1ccccc1)C1=[O+][Cu-3]2([O+]=C(C=Cc3ccccc3...       CI           0
2                   CC(=O)N1c2ccccc2Sc2c1ccc1ccccc21       CI           0
3    Nc1ccc(C=Cc2ccc(N)cc2S(=O)(=O)O)c(S(=O)(=O)O)c1       CI           0


In [18]:
prob_list_for_smile_data = [ str(smile_to_hiv_prob(i,best_model).item()) for i in smiles_data]

print(prob_list_for_smile_data)
print(activity_data)

['0.4207727611064911', '0.7503854036331177', '0.007145126350224018', '0.0018831249326467514']
['CA', 'CA', 'CI', 'CI']


In [ ]:
from captum.attr import Saliency, IntegratedGradients

def model_forward(edge_mask, data):
    batch = torch.zeros(data.x.shape[0], dtype=int).to(device)
    out = model(data.x, data.edge_index, batch, edge_mask)
    return out


def explain(method, data, target=0):
    input_mask = torch.ones(data.edge_index.shape[1]).requires_grad_(True).to(device)
    if method == 'ig':
        ig = IntegratedGradients(model_forward)
        mask = ig.attribute(input_mask, target=target,
                            additional_forward_args=(data,),
                            internal_batch_size=data.edge_index.shape[1])
    elif method == 'saliency':
        saliency = Saliency(model_forward)
        mask = saliency.attribute(input_mask, target=target,
                                  additional_forward_args=(data,))
    else:
        raise Exception('Unknown explanation method')

    edge_mask = np.abs(mask.cpu().detach().numpy())
    if edge_mask.max() > 0:  # avoid division by zero
        edge_mask = edge_mask / edge_mask.max()
    return edge_mask

TypeError: forward_with_graph() takes 2 positional arguments but 3 were given

In [61]:
smiles_data[1]

'O=S(=O)(O)c1cc2ccc1ccc1ccc(cc1S(=O)(=O)O)nnc1ccc(ccc3ccc(cc3S(=O)(=O)O)nn2)c(S(=O)(=O)O)c1'

In [ ]:
import torch
from captum.attr import IntegratedGradients
from torch_geometric.data import DataLoader

def model_forward(single_molecule):
    return model(single_molecule)

graph_data_sample = smiles2graph(smiles_data[1])

single_molecule_like = Data(
   x=torch.tensor(graph_data_sample['node_feat']),
   edge_index=torch.tensor(graph_data_sample['edge_index']),
   edge_attr=torch.tensor(graph_data_sample['edge_feat']),
   num_nodes=torch.tensor(graph_data_sample['num_nodes']) )

single_molecule_like = single_molecule_like.to(device) # device can be 'cuda' or 'cpu'
# best_model.eval()

ig = IntegratedGradients(model_forward)

for data in train_loader:
    data = data.to(device)  # Move data to the appropriate device
    x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr

    # Compute attributions
    attributions, delta = ig.attribute(
        inputs=(x, edge_index, edge_attr),
        target=data.y,
        return_convergence_delta=True
    )

    # Print or visualize the attributions
    print("Attributions:", attributions)
    print("Delta:", delta)
    break

    # use single_molecule_like with best model
# with torch.no_grad():
# logits = best_model(single_molecule_like)
# probabilities = torch.sigmoid(logits)

In [72]:
activity_data[1]

'CA'

In [73]:
hiv_data['HIV_active'][1]

np.int64(0)

In [75]:
graph_data_sample

{'edge_index': array([[ 0,  1,  1,  2,  1,  3,  1,  4,  4,  5,  5,  6,  6,  7,  7,  8,
          8,  9,  9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14, 15, 15, 16,
         16, 17, 17, 18, 18, 19, 18, 20, 18, 21, 15, 22, 22, 23, 23, 24,
         24, 25, 25, 26, 26, 27, 27, 28, 28, 29, 29, 30, 30, 31, 31, 32,
         32, 33, 33, 34, 34, 35, 35, 36, 36, 37, 36, 38, 36, 39, 33, 40,
         40, 41, 27, 42, 42, 43, 43, 44, 43, 45, 43, 46, 42, 47,  9,  4,
         17, 12, 47, 24, 41,  6, 35, 30],
        [ 1,  0,  2,  1,  3,  1,  4,  1,  5,  4,  6,  5,  7,  6,  8,  7,
          9,  8, 10,  9, 11, 10, 12, 11, 13, 12, 14, 13, 15, 14, 16, 15,
         17, 16, 18, 17, 19, 18, 20, 18, 21, 18, 22, 15, 23, 22, 24, 23,
         25, 24, 26, 25, 27, 26, 28, 27, 29, 28, 30, 29, 31, 30, 32, 31,
         33, 32, 34, 33, 35, 34, 36, 35, 37, 36, 38, 36, 39, 36, 40, 33,
         41, 40, 42, 27, 43, 42, 44, 43, 45, 43, 46, 43, 47, 42,  4,  9,
         12, 17, 24, 47,  6, 41, 30, 35]]),
 'edge_feat': array([[1,

In [85]:
single_molecule_like.batch

tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,
         2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
         2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
         3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  4,  4,
         4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
         4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  5,  5,  5,  5,  5,  5,  5,
         5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  6,  6,
         6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
         6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  7,  7,  7,  7,  7,  7,  7,
         7,  7,  7,  8,  8,  8,  8,  8, 

In [116]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

# !pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
# !pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
# !pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
# !pip install -q captum

# Helper function for visualization.
%matplotlib inline
import matplotlib.pyplot as plt
from torch_geometric.loader import DataLoader
from torch_geometric.datasets import TUDataset
import networkx as nx
import numpy as np

from torch_geometric.utils import to_networkx

2.5.1


In [129]:
import networkx as nx
import numpy as np

from torch_geometric.utils import to_networkx


def draw_molecule(g, edge_mask=None, draw_edge_labels=False):
    g = g.copy().to_undirected()
    node_labels = {}
    for u, data in g.nodes(data=True):
        node_labels[u] = data['name']
    pos = nx.planar_layout(g)
    pos = nx.spring_layout(g, pos=pos)
    if edge_mask is None:
        edge_color = 'black'
        widths = None
    else:
        edge_color = [edge_mask[(u, v)] for u, v in g.edges()]
        widths = [x * 10 for x in edge_color]
    nx.draw(g, pos=pos, labels=node_labels, width=widths,
            edge_color=edge_color, edge_cmap=plt.cm.Blues,
            node_color='azure')
    
    if draw_edge_labels and edge_mask is not None:
        edge_labels = {k: ('%.2f' % v) for k, v in edge_mask.items()}    
        nx.draw_networkx_edge_labels(g, pos, edge_labels=edge_labels,
                                    font_color='red')
    plt.show()


def to_molecule(data):
    ATOM_MAP = ['C', 'O', 'Cl', 'H', 'N', 'F',
                'Br', 'S', 'P', 'I', 'Na', 'K', 'Li', 'Ca']
    g = to_networkx(data, node_attrs=['x'])
    for u, data in g.nodes(data=True):
        data['name'] = ATOM_MAP[data['x'].index(1.0)]
        del data['x']
    return g


In [130]:
import random

data = random.choice([t for t in train_sample])
mol = to_molecule(data)
plt.figure(figsize=(10, 5))
draw_molecule(mol)

ValueError: 1.0 is not in list

In [127]:

def to_molecule(data):
    ATOM_MAP = ['C', 'O', 'Cl', 'H', 'N', 'F',
                'Br', 'S', 'P', 'I', 'Na', 'K', 'Li', 'Ca']
    g = to_networkx(data, node_attrs=['x'])
    for u, data in g.nodes(data=True):
        data['name'] = ATOM_MAP[data['x'].index(1.0)]
        del data['x']
    return g

In [125]:

k = to_networkx(single_molecule_like, node_attrs=['x'])
knodes = k.nodes(data=True)

In [128]:

single_molecule_like = Data(
   x=torch.tensor(graph_data_sample['node_feat']),
   edge_index=torch.tensor(graph_data_sample['edge_index']),
   edge_attr=torch.tensor(graph_data_sample['edge_feat']),
   num_nodes=torch.tensor(graph_data_sample['num_nodes']) )

# single_molec ule_like = single_molecule_like.to(device) # device can be 'cuda' or 'cpu'

In [122]:
to_molecule(single_molecule_like)

ValueError: 1.0 is not in list

In [ ]:
#####


######

In [122]:
train_sample[1]

Data(x=[39, 9], edge_index=[2, 88], edge_attr=[88, 3], y=[1, 1], num_nodes=39)

In [35]:
random_sample = data_list[1:2]


# 70/30 spilt train/valid

random_dataset = MoleculeDataset(random_sample)

Processing...
Done!


In [58]:
import networkx as nx
import numpy as np

from torch_geometric.utils import to_networkx


def draw_molecule(g, edge_mask=None, draw_edge_labels=False):
    g = g.copy().to_undirected()
    node_labels = {}
    for u, data in g.nodes(data=True):
        node_labels[u] = data['name']
    pos = nx.planar_layout(g)
    pos = nx.spring_layout(g, pos=pos)
    if edge_mask is None:
        edge_color = 'black'
        widths = None
    else:
        edge_color = [edge_mask[(u, v)] for u, v in g.edges()]
        widths = [x * 10 for x in edge_color]
    nx.draw(g, pos=pos, labels=node_labels, width=widths,
            edge_color=edge_color, edge_cmap=plt.cm.Blues,
            node_color='azure')
    
    if draw_edge_labels and edge_mask is not None:
        edge_labels = {k: ('%.2f' % v) for k, v in edge_mask.items()}    
        nx.draw_networkx_edge_labels(g, pos, edge_labels=edge_labels,
                                    font_color='red')
    plt.show()


def to_molecule(data):
    ATOM_MAP = ['C', 'O', 'Cl', 'H', 'N', 'F',
                'Br', 'S', 'P', 'I', 'Na', 'K', 'Li', 'Ca']
    g = to_networkx(data, node_attrs=['x'])
    for u, data in g.nodes(data=True):
        data['name'] = ATOM_MAP[data['x'].index(1.0)]
        del data['x']
    return g


In [112]:
from captum.attr import Saliency, IntegratedGradients

def model_forward(edge_mask, data):
    batch = torch.zeros(data.x.shape[0], dtype=int).to(device)
    out = model(data.x, data.edge_index, batch, edge_mask)
    return out

In [113]:

def explain(method, data, target=0):
    input_mask = torch.ones(data.edge_index.shape[1]).requires_grad_(True).to(device)
    if method == 'ig':
        ig = IntegratedGradients(model_forward)
        mask = ig.attribute(input_mask, target=target,
                            additional_forward_args=(data,),
                            internal_batch_size=data.edge_index.shape[1])
    elif method == 'saliency':
        saliency = Saliency(model_forward)
        mask = saliency.attribute(input_mask, target=target,
                                  additional_forward_args=(data,))
    else:
        raise Exception('Unknown explanation method')

    edge_mask = np.abs(mask.cpu().detach().numpy())
    if edge_mask.max() > 0:  # avoid division by zero
        edge_mask = edge_mask / edge_mask.max()
    return edge_mask

In [125]:
input_mask = torch.ones(data.edge_index.shape[1]).requires_grad_(True).to(device)
ig = IntegratedGradients(model_forward)

In [ ]:
/help
mask = ig.attribute(input_mask, target=0,
                            additional_forward_args=(data,))

Welcome to Python 3.12's help utility! If this is your first time using
Python, you should definitely check out the tutorial at
https://docs.python.org/3.12/tutorial/.

Enter the name of any module, keyword, or topic to get help on writing
Python programs and using Python modules.  To get a list of available
modules, keywords, symbols, or topics, enter "modules", "keywords",
"symbols", or "topics".

Each module also comes with a one-line summary of what it does; to list
the modules whose name or summary contain a given string such as "spam",
enter "modules spam".

To quit this help utility and return to the interpreter,
enter "q" or "quit".



In [ ]:

mask = ig.attribute(input_mask, target=0,
                            additional_forward_args=(data,))

TypeError: GINGraphPropertyModel.forward() takes 2 positional arguments but 5 were given

In [ ]:

mask = ig.attribute(input_mask, target=0,
                            additional_forward_args=(data,))

TypeError: GINGraphPropertyModel.forward() takes 2 positional arguments but 5 were given

AssertionError: baseline input argument must be either a torch.Tensor or a number             however <class 'torch_geometric.data.data.Data'> detected

In [ ]:


mask = ig.attribute(input_mask, target=0,
                            additional_forward_args=(data,),
                            internal_batch_size=data.edge_index.shape[1])